<a href="https://colab.research.google.com/github/cihanozluk/textsummarizationturkish/blob/main/TextRank_sum_turkish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference



https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
from nltk.translate.bleu_score import sentence_bleu


In [ ]:

# Pre-processing steps



def sentence_similarity(sent1, sent2, stopwords=None):
  if stopwords is None:
    stopwords = []

  sent1 = [w.lower() for w in sent1]
  sent2 = [w.lower() for w in sent2]

  all_words = list(set(sent1 + sent2))

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)

  # Build the vectors
  for w in sent1:
    if w in stopwords:
      continue
    vector1[all_words.index(w)] += 1

  for w in sent2:
    if w in stopwords:
      continue
    vector2[all_words.index(w)] += 1

  return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
  # Create an empty similarity matrix
  similarity_matrix = np.zeros((len(sentences), len(sentences)))

  for idx1 in range(len(sentences)):
    for idx2 in range(len(sentences)):
      if idx1 == idx2: #ignore if both are same sentences
        continue
      similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

  return similarity_matrix


In [ ]:

# Summarization steps

def generate_summary(text, top_n=5, reference_summaries=None):
  import nltk
  nltk.download('stopwords')
  stop_words = stopwords.words('turkish')
  stop_words.append("bir")
  summarize_text = ''

  # Step 1 - Read text and split it
  print(text)
  sentences = nltk.sent_tokenize(text)

  # Step 2 - Generate Similary Martix across sentences
  sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

  # Step 3 - Rank sentences in similarity martix
  %pip install networkx==2.6.3 
  import networkx as nx
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
  scores = nx.pagerank(sentence_similarity_graph)

  # Step 4 - Sort the rank and pick top sentences
  
  ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
  print("Indexes of top ranked_sentence order are ", ranked_sentence)    

  for i in range(top_n):
      summarize_text=ranked_sentence[i][1]

    # Step 5 - Offcourse, output the summarize text
  print(summarize_text)
 
  return summarize_text
 


In [ ]:

%pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'mlsum', 'tu')




df = pd.DataFrame(dataset['train'], columns=dataset['train'].features)

all_news = df.head()
all_news['Summarized_Text'] = ""
for i in range(all_news.shape[0]):
    all_news['Summarized_Text'][i] = generate_summary(repr(all_news['text'][i]),2)

5310.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


Found cached dataset mlsum (/Users/cihanozluk/.cache/huggingface/datasets/mlsum/tu/1.0.0/033c69bbbf1eb198d444f668be75f297cb86251c0671a3d063d1c53c2f231076)


  0%|          | 0/3 [00:00<?, ?it/s]

/var/folders/47/w31q_dds73v4q62tskyf5_gc0000gn/T/ipykernel_24809/1255077638.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_news['Summarized_Text'] = ""
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cihanozluk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...

Note: you may need to restart the kernel to use updated packages.
Indexes of top ranked_sentence order are  [(0.20321813862751975, 'Görgü tanıkları, saldırganların en az 15 kişi olduğunu belirtti"'), (0.2030958127555766, '"Meksika\'nın kuzey sınırında bir grup silahlı kişinin parti yapan kalabalığı taradığı, saldırıda 13 kişinin öldüğü bildirildi.'), (0.2023899103183777, "Saldırıda yaralanan 20'den fazla gencin de ambulanslarla hastaneye kaldırıldıkları belirtildi."), (0.20031213971770145, 'Yaralıların durumlarının netlik kazanmadığı kaydedildi.'), (0.19098399858082465, 'Yetkililer, Ciudad Juarez kentinde dün gece düzenlenen saldırıdan sonra parti verilen evin çevresinde yaşları 15 ila 20 olan 13 kişinin cesetlerinin bulunduğunu söylediler.')]
"Meksika'nın kuzey sınırında bir grup silahlı kişinin parti yapan kalabalığı taradığı, saldırıda 13 kişinin öldüğü bildirildi.
'Fenerbahçe Teknik Direktörü Christoph Daum, Sivasspor karşısında oynayan futbolcuların bireysel ve takım olarak çok iy

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cihanozluk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cihanozluk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
5360.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.
Indexes of top ranked_sentence order are  [(0.03546840986384635, 'Maç sonrası bir değerlendirme yapan Fenerbahçe Teknik Direktörü Daum, takım halinde iyi oynadıklarını belirtti.'), (0.03537251028341869, "'Fenerbahçe Teknik Direktörü Christoph Daum, Sivasspor karşısında oynayan futbolcuların bireysel ve takım olarak çok iyi olduklarını söyledi."), (0.035313063491952, 'Defansta iyi organize olduk ve ilerde iyi ataklar gördük.'), (0.03509035888348205, 'Eksik futbolcular nedeniyle kendileri için Sivasspor maçının çok önem kazandığını belirten Uğur Boral da, takım halinde iyi oynadıklarını ve farklı kazandıklarını söyledi.'), (0.03506595670813419, 'Yedikleri tek gole üzüldüklerini ifade eden Daum, "Kendi yaptığımız bir hatadan dolayı gol yedik.'), (0.034803561641795, 'Böyle ilk yarıdaki gibi oynarlarsa eminim Sivassphor da ilerdeki maçlarda istedikleri puanları alırlar.'), (0.034684939279458996, "5-1\\'lik yenilginin bir anla

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cihanozluk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cihanozluk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
5366.67s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.
Indexes of top ranked_sentence order are  [(0.10311905705271095, "Daimi Konsey açıklamasında, üyelerin, İKT Genel Sekreteri Ekmelettin İhsanoğlu'nun konunun tüm dünyaya duyurulması için gösterdiği yoğun çabaları takdirle karşıladığı, bu çabaların desteklenmeye devam edileceği bildirildi."), (0.10297981917629448, "İsviçre'deki minare krizinin görüşüldüğü ve 57 üye ülke temsilcilerinin katıldığı toplantıdan sonra yapılan açıklamada, bir kez daha İsviçre hükümetinin attığı bu adımdan duyulan üzüntü ve endişe dile getirildi."), (0.10251462797925727, 'Daimi konsey üyelerinin yaptığı toplantıdan sonra yapılan açıklamada, İsviçre hükümetinin attığı bu adımın, demokrasi, insan hakları, dini özgürlükler gibi kavramların tamamıyla çeliştiğine vurgu yapıldı.'), (0.1019600677088238, "İKT Genel Sekreteri Ekmeleddin İhsanoğlu, AA'ya yaptığı açıklamada, İsviçre hükümetinin aldığı bu kararın kendileri için büyük bir sürpriz olduğunu bel

In [ ]:
print(all_news)

                                                text  \
0  Araç sahipleri için pahalı benzine karşı yapıl...   
1  İki insanın kaderini birleştiren olay, Adana S...   
2  Meksika'nın kuzey sınırında bir grup silahlı k...   
3  Fenerbahçe Teknik Direktörü Christoph Daum, Si...   
4  İslam Konferansı Teşkilatı (İKT) Daimi Konsey ...   

                                             summary    topic  \
0  Benzinin litresi 4 liraya yaklaştı. Aslında ar...  unknown   
1  Seyhan Nehri"ne düştüğünü gören genç öğretmen,...  unknown   
2  Meksika'da parti düzenlenen bir eve yapılan sa...  unknown   
3  Fenerbahçe Teknik Direktörü Christoph Daum, Si...  unknown   
4  İslam Konferansı'nda 57 ülkenin temsilcisi İsv...  unknown   

                                                 url  \
0  https://www.internethaber.com/aracinizda-yuzde...   
1  https://www.internethaber.com/kahraman-ogretme...   
2  https://www.internethaber.com/parti-yapanlari-...   
3  https://www.internethaber.com/daum-oynanan-oy

In [ ]:
def calculate_bleu(generated_summary, reference_summaries):
  
  # Convert the summary and reference summaries to lists of sentences
  generated_sentences = generated_summary
  print(generated_sentences)
  
  reference_sentences = reference_summaries
  print(reference_sentences)
  
  # Calculate the BLEU score
  print('Individual 1-gram: %f' % sentence_bleu(reference_sentences, generated_sentences, weights=(1, 0, 0, 0)))
  print('Individual 2-gram: %f' % sentence_bleu(reference_sentences, generated_sentences, weights=(0, 1, 0, 0)))
  print('Individual 3-gram: %f' % sentence_bleu(reference_sentences, generated_sentences, weights=(0, 0, 1, 0)))
  print('Individual 4-gram: %f' % sentence_bleu(reference_sentences, generated_sentences, weights=(0, 0, 0, 1)))

  score = sentence_bleu(reference_sentences, generated_sentences, weights=(1, 0, 0, 0))
  print(score)

  return score

In [ ]:
for i in range(all_news.shape[0]):
    print("{:.2f}".format(calculate_bleu(all_news['Summarized_Text'][i],all_news['summary'][i])))

Uygun motor yağı kullanımı ve temiz yakıt, özellikle filo araçlarına ve kurumlara ciddi tasarruf sağlıyor.
Benzinin litresi 4 liraya yaklaştı. Aslında aracınızda çok küçük bir dikkatle akaryakıtta yüzde 30 kar edebilirsiniz.
Individual 1-gram: 0.169811
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000
0.169811320754717
0.17
Zaten havuza yüzmeye gidiyordum, kısmet nehirde yüzmekmiş, oraya atladık” diyerek anlattı.
Seyhan Nehri"ne düştüğünü gören genç öğretmen, hemen nehre atlayıp hasta kadını kurtardı.
Individual 1-gram: 0.233333
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000
0.23333333333333334
0.23
"Meksika'nın kuzey sınırında bir grup silahlı kişinin parti yapan kalabalığı taradığı, saldırıda 13 kişinin öldüğü bildirildi.
Meksika'da parti düzenlenen bir eve yapılan saldırıda ilk belirlemelere göre 13 kişi yaşamını yitirdi.
Individual 1-gram: 0.198413
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individu